In [141]:
import os
import sys
import json

import elasticsearch
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import torch

from datasets import (
    load_from_disk)

In [142]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [143]:
es.ping()


/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


True

In [232]:
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_mixed',
          'filter': [ 'shingle' ]
          }
        },
      "tokenizer": {
        "nori_none": {
          "type": "nori_tokenizer",
          "decompound_mode": "none"
        },
        "nori_discard": {
          "type": "nori_tokenizer",
          "decompound_mode": "discard"
        },
        "nori_mixed": {
          "type": "nori_tokenizer",
          "decompound_mode": "mixed"
        }
      }
    },
  },
  'mappings':{
    'properties':{
      'text':{
        'type':'text',
        'analyzer': 'nori_analyzer'
      }
    }
  }
}


In [219]:
es.indices.delete(index='test', ignore=[400, 404])

{'acknowledged': True}

In [220]:
es.indices.create(index='test', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_30246/2421981795.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='test', ignore=[400,404], body=CONFIG)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test'}

In [212]:
DOCS = {
    1: {"content": "가, 가까스로, 가령, 각, 각각, 각자, 각종, 갖고말하자면, 이건 될까 안될까"},
    2: {"content": 'The quick brown fox jumps over the lazy dog.'},
    }

In [221]:
DOCS = {
    1: {"title": "My Love",
        "content": "사랑해 그 말은 무엇보다 아픈 말 숨죽여서 하는 말 이젠 하기 힘든 말"
        },
    2: {"title": "듣고있나요",
        "content": "끝내 우린 스쳐가나요 기억넘어 서로를 지워야하나요 내게 사랑이 준 깊은 상처는 어떻게 견디며 살아야하는지 매일 아픈 그리움속에 가슴 텅 빈채 살아도 그대를 사랑했던일 그것만은 죽어도 나 후회하지않아요"
        },
    3: {"title": "인연",
        "content": "눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 않아 견디기 힘들어 운명같은 우연을"
        },
    4: {"title": "말리 꽃",
        "content": ["얼마나 더 견뎌야 하는지 짙은 어둠을 헤메고 있어","내가 바란 꿈이라는 것은 없는걸까", "더 이상은 견딜수 없는 것"]
        },
    5: {"title": "그런 사람 또 없습니다",
        "content": "천번 번이고 다시 태어난대도 그런 사람 또 없을테죠 슬픈 내삶을 따뜻하게 해준 잠 고마운 사람입니다"
        }
}

In [222]:
import time
for doc_id, doc in DOCS.items():
    es.index(index='test', id=doc_id, body=doc)
    time.sleep(1)

/tmp/ipykernel_30246/3642452819.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='test', id=doc_id, body=doc)


In [223]:
es.indices.analyze(index='test', body={"analyzer" : "nori_analyzer", "text" : "가, 가까스로, 가령, 각, 각각, 각자"})


{'tokens': [{'token': '가',
   'start_offset': 0,
   'end_offset': 1,
   'type': 'word',
   'position': 0},
  {'token': '가 가까스로',
   'start_offset': 0,
   'end_offset': 7,
   'type': 'shingle',
   'position': 0,
   'positionLength': 2},
  {'token': '가까스로',
   'start_offset': 3,
   'end_offset': 7,
   'type': 'word',
   'position': 1},
  {'token': '가까스로 가령',
   'start_offset': 3,
   'end_offset': 11,
   'type': 'shingle',
   'position': 1,
   'positionLength': 2},
  {'token': '가령',
   'start_offset': 9,
   'end_offset': 11,
   'type': 'word',
   'position': 2},
  {'token': '가령 각',
   'start_offset': 9,
   'end_offset': 14,
   'type': 'shingle',
   'position': 2,
   'positionLength': 2},
  {'token': '각',
   'start_offset': 13,
   'end_offset': 14,
   'type': 'word',
   'position': 3},
  {'token': '각 각각',
   'start_offset': 13,
   'end_offset': 18,
   'type': 'shingle',
   'position': 3,
   'positionLength': 2},
  {'token': '각각',
   'start_offset': 16,
   'end_offset': 18,
   'type': 'word

In [224]:
query = "사랑하지만 힘들어 죽겠네"
res = es.search(index='test', q=query, size=10)
pprint.pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '3',
                    '_index': 'test',
                    '_score': 1.2458736,
                    '_source': {'content': '눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 '
                                           '눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 '
                                           '않아 견디기 힘들어 운명같은 우연을',
                                'title': '인연'},
                    '_type': '_doc'}],
          'max_score': 1.2458736,
          'total': {'relation': 'eq', 'value': 1}},
 'timed_out': False,
 'took': 1}


In [208]:
es.search(index='test', q="사랑하지만 힘들어 죽겠네, 가, 가까스로, 가령, 각, 각각?", size =1)
# es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':q1}}})

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.3977623,
  'hits': [{'_index': 'test',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.3977623,
    '_source': {'title': '인연',
     'content': '눈을 떠 바라보아요 그댄 정말 가셨나요 단 한번 보내준 그대 눈빛은 날 사랑했나요 또 다른 사랑이 와도 이젠 쉽게 허락되진 않아 견디기 힘들어 운명같은 우연을'}}]}}

In [49]:
res = es.indices.get_alias("*")
for Name in res:
    print(Name)

test
wiki_documents


/tmp/ipykernel_20892/3218817535.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.indices.get_alias("*")


In [6]:
es.indices.delete(index='toy_index', ignore=[400, 404])

{'acknowledged': True}

In [7]:
with open('/opt/ml/data/wikipedia_documents.json') as f :
    data = json.load(f)

In [8]:
next(iter(data.items()))

('0',
 {'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
  'corpus_source': '위키피디아',
  'url': 'TODO',
  'domain': None,
  'title': '나라 목록',
  'author': None,
  'html': None,
  'document_id': 0})

In [9]:
df = pd.read_json('/opt/ml/data/wikipedia_documents.json').T

In [10]:
df.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [52]:
df = pd.read_json('/opt/ml/data/wikipedia_documents.json').T
df = df.drop_duplicates(subset=['text'])
# df = df.to_dict('records')

In [64]:
len(df)//100, len(df)%100

len(df[:100*(len(df)//100)])

56700

In [51]:
df = df.drop_duplicates(subset=['text']).reset_index()
df.nunique()

index            56737
text             56737
corpus_source        1
url                  1
domain               0
title            31726
author               0
html                 0
document_id      56737
dtype: int64

In [52]:
len(df)

56737

In [53]:
df_rec_dict = df.to_dict('records')

In [54]:
def generator(df):
  for c, line in enumerate(df):
    yield {
      '_index': 'wiki_documents',
      '_type': '_doc',
      '_id': line.get('document_id', None),
      '_source': {
        'title': line.get('title', ''),
        'text': line.get('text', '')
      }
    }
  raise StopIteration

In [55]:
CONFIG = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori_tokenizer'
        }
      },
    }
  },
  'mappings':{
    'properties':{
      'title':{
        'type':'text'
      },
      'text':{
        'type':'text',
        "analyzer": "nori_analyzer"
      }
    }
  }
}

In [56]:
es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)

/tmp/ipykernel_18337/2280445778.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index='wiki_documents', ignore=[400,404], body=CONFIG)
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [wiki_documents/lwi98_RFRrCPTBRGTlIrRg] already exists',
    'index_uuid': 'lwi98_RFRrCPTBRGTlIrRg',
    'index': 'wiki_documents'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [wiki_documents/lwi98_RFRrCPTBRGTlIrRg] already exists',
  'index_uuid': 'lwi98_RFRrCPTBRGTlIrRg',
  'index': 'wiki_documents'},
 'status': 400}

In [57]:
gen = generator(df_rec_dict)

In [58]:
try:
  res = helpers.bulk(es, gen, chunk_size = 1)
  # print(res)
except Exception as e:
  print(e)
  print('Done')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


generator raised StopIteration
Done


In [66]:
res

(0, [])

In [67]:
es.count(index = 'wiki_documents')

{'count': 56736,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [68]:
es.get(index='wiki_documents', id=60000)

{'_index': 'wiki_documents',
 '_type': '_doc',
 '_id': '60000',
 '_version': 2,
 '_seq_no': 113928,
 '_primary_term': 1,
 'found': True,
 '_source': {'title': '설원 (책)',
  'text': '≪설원≫은 유가의 정치사상과 윤리도덕의 관념을 깊이 반영했다. 이것은 유향이 유가사상가이자 학자로서 자신의 책무를 수행하려 했던 결과다. 또한 왕조가 쇠퇴하는 추세뿐 아니라 지도층의 부정부패와 사치를 목격하고서, 제국을 바로잡아 보려는 노력을 반영한 것이기도 하다. 그래서 순임금, 우임금으로부터 진한(秦漢)에 이르기까지 여러 인물의 언행이나 사건 또는 일화를 모아 정치의 흥망을 엿볼 수 있는 역사의 거울을 제공했다. 이를 통해 군주와 신하들을 권면하고 조정을 정돈하며, 당시의 폐단을 바로잡으려 했던 것이다. 이에 유향은 각 고사를 통해서 제왕, 장상(將相), 사대부 들에게 어떻게 해야 바른 군주, 바른 신하, 바른 백성이 될 것인가, 그리고 어떻게 해야 제국을 일으키고 백성을 다스리며 외교를 처리할 것인가를 제시했다.\n\n이렇게 유향이 모은 이야기들에는 백성이 치국의 근본이 되어야 한다든가, 세금을 경감해야 한다든가, 욕구를 억제하고 사치를 경계해야 한다든가, 통치자는 현신과 간신을 구별해야 한다든가, 현사를 높이고 예우해야 한다는 주장 등, 통치자라면 누구나 따라야 할 통치의 모든 덕목이 담겨 있다.'}}

In [89]:
tmp = load_from_disk('/opt/ml/data/test_dataset')
tmp

DatasetDict({
    validation: Dataset({
        features: ['id', 'question'],
        num_rows: 600
    })
})

In [64]:
datasets = load_from_disk('/opt/ml/data/train_dataset') 
train_df = pd.DataFrame(datasets["train"])

In [75]:
q1 = train_df.loc[0,'question']

In [76]:
es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':q1}}})

/tmp/ipykernel_18337/1279345985.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index='wiki_documents', body={'size':1, 'query':{'match':{'text':q1}}})
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'took': 16,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 37.232613,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '10598',
    '_score': 37.232613,
    '_source': {'title': '내각',
     'text': '내각(內閣, cabinet)은 행정부의 주요 각료들로 구성되는 국가의 주요기관이다.\n\n의원내각제에서 내각은 수상과 여러 장관으로 조직되는 합의체로, 국가의 행정권을 담당하고 국회에 대한 연대책임을 갖는다. 의원내각제에 있어서 내각은 국가행정의 최고기관인 한편 국민이 구성시키는 의회에 의하여 철저히 견제되어 의회민주주의 체제를 이룬다.\n\n그 직접적 유래는 영국에서 국왕의 정치를 자문하던 추밀원에서 찾을 수 있다. 특히 내각은 추밀원의 일개 회의에서 시작하였다가 권한이 집중되어 분리된 기관으로, 이후 국왕의 실권이 사라지고 일명 웨스트민스터 시스템으로 불리는 의원내각제가 성립하면서 의회에 의한 민주적 행정부를 이루게 되었다.\n\n한편 국가원수에게 대부분의 권력이 집중되는 대통령중심제와 군주제에서 내각은 원칙적으로 의결권이 없거나 의결의 구속력이 없는 보좌기관에 불과한 경우가 많다.(예: 대한민국의 국무회의)\n\n대한민국은 국무회의가 내각에 속하며 권한이 대통령에 비해 제한적이다. 과거 왕조시대 때는 고려시대의 중서문하성, 중추원, 육부 또는 조선시대의 의정부와 육조가 내각과 비슷한 성향을 지니고 있었다.'}}]}}

In [83]:
es.indices.get_mapping(index='wiki_documents')

{'wiki_documents': {'mappings': {'properties': {'text': {'type': 'text',
     'analyzer': 'nori_analyzer'},
    'title': {'type': 'text'}}}}}

In [84]:
settings = {
  'settings':{
    'number_of_shards':1,
    'number_of_replicas':1,
    'analysis':{
      'analyzer': {
        'nori_analyzer':{
          'type': 'custom',
          'tokenizer': 'nori'
        }
      }
    }
  }
}

In [86]:
es.indices.analyze(
  body={
    'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.',
    'analyzer':'nori',
  }
)

{'tokens': [{'token': '문서',
   'start_offset': 2,
   'end_offset': 4,
   'type': 'word',
   'position': 1},
  {'token': '나라',
   'start_offset': 6,
   'end_offset': 8,
   'type': 'word',
   'position': 3},
  {'token': '목록',
   'start_offset': 9,
   'end_offset': 11,
   'type': 'word',
   'position': 4},
  {'token': '이',
   'start_offset': 11,
   'end_offset': 12,
   'type': 'word',
   'position': 5},
  {'token': '세계',
   'start_offset': 17,
   'end_offset': 19,
   'type': 'word',
   'position': 8},
  {'token': '206',
   'start_offset': 20,
   'end_offset': 23,
   'type': 'word',
   'position': 9},
  {'token': '개',
   'start_offset': 23,
   'end_offset': 24,
   'type': 'word',
   'position': 10},
  {'token': '나라',
   'start_offset': 25,
   'end_offset': 27,
   'type': 'word',
   'position': 11},
  {'token': '현황',
   'start_offset': 31,
   'end_offset': 33,
   'type': 'word',
   'position': 14},
  {'token': '주권',
   'start_offset': 35,
   'end_offset': 37,
   'type': 'word',
   'position

In [87]:
es.search(
  index='wiki_documents', 
  body={
    'query':{
      'match':{
        'text':'용병회사의 경기가 좋아진 것은 무엇이 끝난 이후부터인가?',
        # 'tokenizer': 'nori'
      }
    }
  },
  size=3
)

/tmp/ipykernel_18337/1677013031.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(


{'took': 36,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 21.451807,
  'hits': [{'_index': 'wiki_documents',
    '_type': '_doc',
    '_id': '20081',
    '_score': 21.451807,
    '_source': {'title': '크리스 니코스키',
     'text': 'SK에서 방출된 이후 그는 일본을 여행하며 잠시 신변을 정리하고 있었다. 그 때 좌완 선발 투수가 필요하여 니코스키에게 관심을 두고 있었던 두산 베어스가 SK 와이번스에 계약 양도를 신청하여 곧바로 맷 왓슨의 대체 용병으로 두산 베어스로 이적했고 두산 베어스의 선발진에 합류하여 비로소 첫 승을 따 냈으며  승리를 대부분 한화 이글스전에서 기록하여 한화 이글스에는 강력한 천적으로 자리매김했다.  그러나 예전부터 주로 원포인트 및 중간계투로 뛰다가 선발로 전환한 탓에 선발로는 아직 익숙하지 않아 3회까지 투구수가 70개 가까이 되는 경기가 대부분이라는 단점이 있었다.  2009년 9월 13일 KIA 타이거즈전에서 비로소 초반에 급격하게 늘어나는 투구수를 극복하여 승리를 따 내기도 했다.  두산 베어스 합류 후 12경기에 더 나서고 2009년 정규 시즌을 4승 8패, 평균 자책 3.78로 마무리했다. 완전한 선발 전업 후 후반기에 조금씩 나아지는 모습을 보였으나, 잠실야구장에서 롯데 자이언츠와 맞붙었던 2009년 9월 29일 준 플레이오프 1차전에서 무실점으로 3이닝을 호투하던 중 갑작스러운 어깨 통증으로 조기 강판당했고, 그 날 두산 베어스는 롯데 자이언츠에게 1차전을 내주었다. 진단 결과 극상근 손상 판정을 받아 포스트 시즌 전력에서 완전히 이탈하고 말았다. 